# **LangChain Memory**

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

We call this ability to store information about past interactions "memory". LangChain provides a lot of utilities for adding memory to a system. These utilities can be used by themselves or incorporated seamlessly into a chain.

## **Building memory into a system**
The two core design decisions in any memory system are:
- How state is stored
- How state is queried

## **What's covered?**
- ConversationBufferMemory
- End-to-end Example: Conversational AI Bot
- Saving and Loading a Chat History
- ChatMessageHistory [Beta]
- ConversationBufferWindowMemory [Beta]
- ConversationSummaryMemory [Beta]

## **ConversationBufferMemory**

Let's take a look at how to use ConversationBufferMemory in chains. ConversationBufferMemory is an extremely simple form of memory that just keeps a list of chat messages in a buffer and passes those into the prompt template.

This memory type can be connected to a conversation. It allows for storing messages and then extracts the messages in a variable.

In [1]:
# Importing ConversationBufferMemory
from langchain.memory import ConversationBufferMemory

# Set up an instance of Conversation buffer memory
memory = ConversationBufferMemory()

In [2]:
# Loading Memory

memory.load_memory_variables({})

{'history': ''}

In [3]:
# Loading memory buffer with history

memory.buffer

''

### **Renaming the memory variable and Adding Messages**

In [4]:
memory = ConversationBufferMemory(memory_key="chat_history")

memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [5]:
memory.load_memory_variables({})

{'chat_history': "Human: hi!\nAI: what's up?"}

In [6]:
memory.buffer

"Human: hi!\nAI: what's up?"

### **By default, memory is returned as a single string. In order to return as a list of messages, you can set `return_messages=True`**

In [7]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [8]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi!'),
  AIMessage(content="what's up?")]}

In [9]:
memory.buffer

[HumanMessage(content='hi!'), AIMessage(content="what's up?")]

## **End-to-end Example: Conversational AI Bot**

<img src="images/memory.png">

### **Steps:**
1. Import Chat Model and Configure the API Key
2. Create Chat Template
3. Initialize the Memory
4. Create a Output Parser
5. Build a Chain
6. Invoke the chain with human_input and chat_history
7. Saving to memory
8. Run Step 6 and 7 in a loop

In [10]:
# Step 1 - Import Chat Model and Configure the API Key

from langchain_openai import ChatOpenAI

# Setup API Key
f = open('keys/.openai_api_key.txt')
OPENAI_API_KEY = f.read()

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [11]:
# Step 2 - Create Chat Template

from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        # The persistent system prompt
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),
        # Creating a chat_history placeholder
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  
        # Human Prompt
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),
    ]
)

In [12]:
# Step 3 - Initialize the Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [13]:
memory.buffer

[]

In [14]:
chat_prompt_template.invoke({"human_input": "Hi", "chat_history": memory.buffer})

ChatPromptValue(messages=[SystemMessage(content='You are a chatbot having a conversation with a human.'), HumanMessage(content='Hi')])

In [15]:
# Step 4 - Create a Output Parser

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [16]:
# Step 5 - Build a Chain

chain = chat_prompt_template | chat_model | output_parser

In [17]:
# Step 6 - Invoke the chain with human_input and chat_history

human_input = "Hi"

response = chain.invoke({"human_input": human_input, "chat_history": memory.buffer})

response

'Hello! How are you today?'

In [18]:
memory.buffer

[]

In [19]:
# Step 7 - Saving to Memory

memory.chat_memory.add_user_message(human_input)
memory.chat_memory.add_ai_message(response)

memory.buffer

[HumanMessage(content='Hi'), AIMessage(content='Hello! How are you today?')]

In [20]:
# Step 8 - Run Step 6 and 7 in a loop

while True:
    human_input = input('Enter your input: ')
    print(f"*User: {human_input}")
    if human_input in ['bye', 'quit', 'exit']:
        break
    response = chain.invoke({"human_input": human_input, "chat_history": memory.buffer})
    print(f"*AI: {response}")

    memory.chat_memory.add_user_message(human_input)
    memory.chat_memory.add_ai_message(response)

Enter your input:  I am good. My name is Kanav.


*User: I am good. My name is Kanav.
*AI: Nice to meet you, Kanav! Is there anything specific you'd like to talk about or do today?


Enter your input:  Can you tell me what kind of tasks can you solve for me?


*User: Can you tell me what kind of tasks can you solve for me?
*AI: I can assist you with a variety of tasks such as answering questions, providing information on a wide range of topics, helping you make decisions, offering suggestions, giving advice, and even just chatting for fun. Feel free to ask me anything you'd like help with!


Enter your input:  That's good to know.


*User: That's good to know.
*AI: I'm here to help whenever you need assistance or just want to chat. Is there anything else you'd like to know or talk about?


Enter your input:  What is my name?


*User: What is my name?
*AI: Your name is Kanav.


Enter your input:  bye


*User: bye


## **Saving a Chat History**

**Let's now learn to save this history on the disk so that whenever we can load the history whenever we chat with our assistant.**

In [21]:
import pickle

chat_history = pickle.dumps(memory)

with open("chats_data/conversation_memory.pkl", "wb") as f:
    f.write(chat_history)

## **Loading a Chat History**

In [22]:
chat_history_loaded = pickle.load(open("chats_data/conversation_memory.pkl", "rb"))

print(chat_history_loaded)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi'), AIMessage(content='Hello! How are you today?'), HumanMessage(content='I am good. My name is Kanav.'), AIMessage(content="Nice to meet you, Kanav! Is there anything specific you'd like to talk about or do today?"), HumanMessage(content='Can you tell me what kind of tasks can you solve for me?'), AIMessage(content="I can assist you with a variety of tasks such as answering questions, providing information on a wide range of topics, helping you make decisions, offering suggestions, giving advice, and even just chatting for fun. Feel free to ask me anything you'd like help with!"), HumanMessage(content="That's good to know."), AIMessage(content="I'm here to help whenever you need assistance or just want to chat. Is there anything else you'd like to know or talk about?"), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Kanav.')]) return_messages=True memory_key='chat_history'


In [23]:
chat_history_loaded.buffer

[HumanMessage(content='Hi'),
 AIMessage(content='Hello! How are you today?'),
 HumanMessage(content='I am good. My name is Kanav.'),
 AIMessage(content="Nice to meet you, Kanav! Is there anything specific you'd like to talk about or do today?"),
 HumanMessage(content='Can you tell me what kind of tasks can you solve for me?'),
 AIMessage(content="I can assist you with a variety of tasks such as answering questions, providing information on a wide range of topics, helping you make decisions, offering suggestions, giving advice, and even just chatting for fun. Feel free to ask me anything you'd like help with!"),
 HumanMessage(content="That's good to know."),
 AIMessage(content="I'm here to help whenever you need assistance or just want to chat. Is there anything else you'd like to know or talk about?"),
 HumanMessage(content='What is my name?'),
 AIMessage(content='Your name is Kanav.')]

## **ChatMessageHistory**

This allows us to mannualy add the messages to memory.

In [24]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

In [25]:
history.add_user_message("Hello, Nice to meet you.")

In [26]:
history.add_ai_message("Nice to meet you!")

In [27]:
history.messages

[HumanMessage(content='Hello, Nice to meet you.'),
 AIMessage(content='Nice to meet you!')]

## **ConversationBufferWindowMemory [Beta]**

With this we will have an ability to call back just a window of conversation.

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [25]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

In [27]:
# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

# Set up an instance of Conversation buffer window memory
window_memory = ConversationBufferWindowMemory(k=1)
# here, k most recent interactions will be refered to answer the user queries

# Set up an assistant
conversation = ConversationChain(llm=chat_model, memory=window_memory, verbose=True)

In [29]:
conversation.predict(input="Hello, How are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, How are you?
AI:

> Finished chain.


"Hello! I'm doing well, thank you for asking. I've been busy processing information and learning new things. How about you?"

In [30]:
conversation.predict(input="What is the new thing you learned recently?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, How are you?
AI: Hello! I'm doing well, thank you for asking. I've been busy processing information and learning new things. How about you?
Human: What is the new thing you learned recently?
AI:

> Finished chain.


"I recently learned about a new algorithm for image recognition that improves accuracy by 10%. It's called Convolutional Neural Networks with Attention Mechanism (CNN-AM). This algorithm focuses on specific parts of an image to make more accurate predictions. I found it fascinating how it can significantly improve image recognition tasks."

In [31]:
conversation.predict(input="Sounds interesting. Can you teach it to me in a simple term?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is the new thing you learned recently?
AI: I recently learned about a new algorithm for image recognition that improves accuracy by 10%. It's called Convolutional Neural Networks with Attention Mechanism (CNN-AM). This algorithm focuses on specific parts of an image to make more accurate predictions. I found it fascinating how it can significantly improve image recognition tasks.
Human: Sounds interesting. Can you teach it to me in a simple term?
AI:

> Finished chain.


'Sure! In simple terms, the CNN-AM algorithm is like a detective that pays extra attention to important parts of a picture to figure out what the picture is showing. It helps computers recognize images more accurately by focusing on key details.'

In [33]:
print(conversation.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hello, How are you?'), AIMessage(content="Hello! I'm doing well, thank you for asking. I've been busy processing information and learning new things. How about you?"), HumanMessage(content='What is the new thing you learned recently?'), AIMessage(content="I recently learned about a new algorithm for image recognition that improves accuracy by 10%. It's called Convolutional Neural Networks with Attention Mechanism (CNN-AM). This algorithm focuses on specific parts of an image to make more accurate predictions. I found it fascinating how it can significantly improve image recognition tasks."), HumanMessage(content='Sounds interesting. Can you teach it to me in a simple term?'), AIMessage(content='Sure! In simple terms, the CNN-AM algorithm is like a detective that pays extra attention to important parts of a picture to figure out what the picture is showing. It helps computers recognize images more accurately by focusing on k

In [32]:
print(window_memory.buffer)

Human: Sounds interesting. Can you teach it to me in a simple term?
AI: Sure! In simple terms, the CNN-AM algorithm is like a detective that pays extra attention to important parts of a picture to figure out what the picture is showing. It helps computers recognize images more accurately by focusing on key details.


## **ConversationSummaryBufferMemory [Beta]**

It allows us to call a LLM for a summary of all the historical conversation.

`ConversationSummaryBufferMemory` combines the two ideas. It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. It uses token length rather than number of interactions to determine when to flush interactions.

In [37]:
from langchain.memory import ConversationSummaryBufferMemory

summary_memory = ConversationSummaryBufferMemory(llm=chat_model, max_token_limit=100)

conversation = ConversationChain(llm=chat_model, memory=summary_memory)

In [38]:
conversation.predict(input="Give me a study plan for Python Programming for my upcoming Data Science Interview.")

"Sure! For your upcoming Data Science interview, I recommend starting with the basics of Python programming. Here's a suggested study plan:\n\n1. Week 1: Introduction to Python - Learn about Python syntax, data types, variables, and basic operations.\n2. Week 2: Control flow and functions - Dive into if statements, loops, and functions in Python.\n3. Week 3: Data structures - Explore lists, dictionaries, tuples, and sets in Python.\n4. Week 4: File handling and modules - Understand how to read and write files, as well as import modules in Python.\n5. Week 5: Object-oriented programming - Learn about classes, objects, and inheritance in Python.\n6. Week 6: Libraries and frameworks - Familiarize yourself with popular libraries like NumPy, Pandas, and Matplotlib for data manipulation and visualization.\n7. Week 7: Practice coding exercises - Solve coding challenges on platforms like LeetCode or HackerRank to reinforce your skills.\n8. Week 8: Review and mock interviews - Review key concep

In [39]:
conversation.predict(input="Also share for Machine Learning.")

"For Machine Learning, I recommend a 12-week study plan to cover the fundamentals of machine learning algorithms, data preprocessing, model evaluation, feature selection, and tuning hyperparameters. I also suggest diving into specific topics such as regression, classification, clustering, and neural networks. Additionally, it's important to work on real-world projects and participate in Kaggle competitions to apply your knowledge and showcase your skills to potential employers during the interview process. Let me know if you need more specific details or resources for your study plan!"

In [40]:
print(summary_memory.buffer)

[]


In [41]:
print(conversation.memory)

llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000024EA67AC2B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000024EA67AF7C0>, openai_api_key=SecretStr('**********'), openai_proxy='') max_token_limit=100 moving_summary_buffer='The human asks the AI for study plans for Python Programming and Machine Learning for an upcoming Data Science Interview. The AI provides a detailed 8-week study plan for Python and a 12-week study plan for Machine Learning, emphasizing the importance of regular practice, real-world application, and participating in Kaggle competitions to showcase skills.'
